In [1]:
# install dependencies
!pip install -q -U pip
%pip install -q -r requirements.txt

In [2]:
# setup parameters
scratch = "../scratch/"

In [4]:
import numpy as np
import os, random, pathlib, warnings, itertools, math, json
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from PIL import Image
from PIL import ImageEnhance
from shutil import rmtree

import boto3
import botocore
from botocore import UNSIGNED
from botocore.client import Config

from tensorflow.keras import models
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.layers import (
    Dense,
    Flatten,
    GlobalAveragePooling2D,
    Dense,
    Dropout,
)

In [5]:
K.clear_session()

In [6]:
# setup paths for data
dataset = scratch + "Vegetable Images"

train_folder = os.path.join(dataset, "train")
test_folder = os.path.join(dataset, "validation")
validation_folder = os.path.join(dataset, "test")

In [9]:
IMAGE_SIZE = [224, 224]

inception = InceptionV3(
    input_shape=IMAGE_SIZE + [3], weights="imagenet", include_top=False
)

for layer in inception.layers:
    layer.trainable = False

x = inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.2)(x)

prediction = Dense(15, activation="softmax")(x)

model = Model(inputs=inception.input, outputs=prediction)

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()

train_datagen = image.ImageDataGenerator(
    rescale=1.0 / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True
)

test_datagen = image.ImageDataGenerator(rescale=1.0 / 255)

training_set = train_datagen.flow_from_directory(
    train_folder, target_size=(224, 224), batch_size=64, class_mode="categorical"
)

test_set = test_datagen.flow_from_directory(
    test_folder, target_size=(224, 224), batch_size=64, class_mode="categorical"
)

class_map = training_set.class_indices
print(class_map)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_188 (Conv2D)            (None, 111, 111, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_188 (Batch  (None, 111, 111, 32  96         ['conv2d_188[0][0]']             
 Normalization)                 )                                                           

In [10]:
model_metadata = "_inceptionV3_epoch5"
path_to_model = scratch + "model" + model_metadata + ".h5"

In [11]:
r = model.fit(
    training_set,
    validation_data=test_set,
    epochs=5,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set),
)

model.save(path_to_model)

Epoch 1/5


2023-05-26 10:44:05.213037: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-26 10:44:12.000814: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-26 10:44:12.528307: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


235/235 [==============================] - ETA: 0s - loss: 0.4710 - accuracy: 0.8636

2023-05-26 10:48:02.488164: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-26 10:48:06.247767: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


235/235 [==============================] - 278s 1s/step - loss: 0.4710 - accuracy: 0.8636 - val_loss: 0.0543 - val_accuracy: 0.9793
Epoch 2/5
235/235 [==============================] - 279s 1s/step - loss: 0.1194 - accuracy: 0.9623 - val_loss: 0.0305 - val_accuracy: 0.9880
Epoch 3/5
235/235 [==============================] - 261s 1s/step - loss: 0.0743 - accuracy: 0.9756 - val_loss: 0.0226 - val_accuracy: 0.9937
Epoch 4/5
 79/235 [=========>....................] - ETA: 3:14 - loss: 0.0485 - accuracy: 0.9844

In [ ]:
K.clear_session()

print("Model: Loading...")
model = load_model(path_to_model)
print("Model: Loaded")

In [ ]:
category = class_map
print(json.dumps(category, indent=4))


def predict_image(filename, model):
    img_ = image.load_img(filename, target_size=(224, 224))
    img_array = image.img_to_array(img_)
    img_processed = np.expand_dims(img_array, axis=0)
    img_processed /= 255.0

    prediction = model.predict(img_processed)
    index = np.argmax(prediction)

    plt.title("Prediction - {}".format(category[index]))
    plt.imshow(img_array)

In [ ]:
predict_image(os.path.join(validation_folder, "Cauliflower/1064.jpg"), model)

In [ ]:
predict_image(os.path.join(validation_folder, "Bitter_Gourd/1202.jpg"), model)

In [ ]:
predict_image(os.path.join(validation_folder, "Papaya/1266.jpg"), model)